In [33]:
import sys
import os
import random
import collections
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from PIL import Image, ImageFile

import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as T

In [7]:
# seed
def Seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
Seed(2021)    

In [8]:
# DIR
submit_dir = '../input/sartorius-cell-instance-segmentation/sample_submission.csv'
train_dir = '../input/sartorius-cell-instance-segmentation/train'
test_dir = '../input/sartorius-cell-instance-segmentation/test'
train_csv_dir = '../input/sartorius-cell-instance-segmentation/train.csv'

# 모델 정의
NUM_EPOCHS = 12
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset 정의

In [16]:
# utilities
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle : run-length a string formated(start length)
    shape : (height, width) of array to return
    Returns numpy array, 1-mask, 0-background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

def get_transforms(train):
    transforms = []
    transforms.append(ToTensorV2())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
class CellDataset(Dataset):
    def __init__(self, image_dir, df_path, height, width, transforms=None):
        self.image_dir = image_dir
        self.df = pd.read_csv(df_path)
        self.height = height
        self.width = width
        self.image_info = collections.defaultdict(dict)
        self.transforms = transforms
        temp_df = self.df.groupby('id')['annotation'].agg(lambda x: list(x)).reset_index()
        for index, row in temp_df.iterrows():  # image id, annotation 붙이기
            self.image_info[index] = {
                'image_id' : row['id'],
                'image_path' : os.path.join(self.image_dir, row['id'] + '.png'),
                'annotations' : row['annotation']
            }
    
    def __getitem__(self, idx):
        img_path = self.image_info[idx]['image_path']
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = Image.open(img_path).convert('RGB')
        #img = img.resize((self.width, self.height), resample=Image.BILINEAR)
        
        info = self.image_info[idx]
        
        mask = np.zeros((len(info['annotations']), self.width, self.height), dtype=np.uint8)
        labels = []
        
        for m, annotation in enumerate(info['annotations']):
            sub_mask = rle_decode(annotation, (520, 704))
            sub_mask = Image.fromarray(sub_mask)
            #sub_mask = sub_mask.resize((self.width, self.height), resample=Image.BILINEAR)
            sub_mask = np.array(sub_mask) > 0
            mask[m, :, :] = sub_mask
            labels.append(1)
        
        num_objs = len(labels)
        boxes = []
        new_labels = []
        new_masks = []
        
        for i in range(num_objs):  # obj box
            try:
                
        